In [46]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes

def hybrid_rsa_aes_encryption(data, public_key):
    # Generate a random AES key
    aes_key = get_random_bytes(16)

    # Encrypt the data with AES
    cipher_aes = AES.new(aes_key, AES.MODE_EAX)
    ciphertext, tag = cipher_aes.encrypt_and_digest(data)

    # Encrypt the AES key with RSA
    cipher_rsa = PKCS1_OAEP.new(public_key)
    cipher_key = cipher_rsa.encrypt(aes_key)

    return (ciphertext, tag, cipher_aes.nonce, cipher_key)

def hybrid_rsa_aes_decryption(ciphertext, tag, nonce, cipher_key, private_key):
    # Decrypt the AES key with RSA
    cipher_rsa = PKCS1_OAEP.new(private_key)
    aes_key = cipher_rsa.decrypt(cipher_key)

    # Decrypt the data with AES
    cipher_aes = AES.new(aes_key, AES.MODE_EAX, nonce=nonce)
    data = cipher_aes.decrypt_and_verify(ciphertext, tag)

    return data


In [47]:
from Crypto.PublicKey import RSA

# Generate RSA key pair for demonstration purposes
key = RSA.generate(2048)
public_key = key.publickey()
private_key = key

# Original data to be encrypted
original_data = b"This is a confidential message."

# Encryption using the hybrid_rsa_aes_encryption function
encrypted_data = hybrid_rsa_aes_encryption(original_data, public_key)

# Decryption using the hybrid_rsa_aes_decryption function
decrypted_data = hybrid_rsa_aes_decryption(*encrypted_data, private_key)

# Print the results
print("Original Data:", original_data)
print("Encrypted Data:", encrypted_data)
print("Decrypted Data:", decrypted_data)

# Verify if the decryption is successful
assert original_data == decrypted_data, "Decryption failed!"
print("Decryption successful.")


Original Data: b'This is a confidential message.'
Encrypted Data: (b'\x7f\xd1\x0fu\xcd\xbb\xfb\xd8\xd6\xaa>\xe4d\x8b_\xd6d\xe4\x9c\x90\xe3]\xf6\xc9`\xb4\x02\xe4Z5\xac', b'{(\xbc#\x0e\xcd\x96:ma\x1b&\xd3M\xbe\x99', b'4%d\r\x88\x98}v\xbf\xef\x80\xaf\xec\xb6p%', b'\xcc]<\x92b=(.\xfa\xc6\x91\xca\x1b,\xb4\xdat\x89\x01\xfd\xd7\xc5\x18\xc34Z\xa4\x14\xf7\rm\xae.8b\x87G\x8cT\x80\x89C\xcd\x93"\xa6\x11L\x07\x05D\xe2\xfb`"-\x8c\xc9i\x93z1!\xae\x84\x18r\x17\xab\xd1\xfc\x00a\xec\x83\xedx\xdba\x80\x98\xd9\xa7\xd4\x9c;\xe3\x85\xe0\xa2~\xa1\x8a9\xc4L\xe5\xb2z\x08\xac\xc0d}\xcc\xb6\x1f\x93\x10rJ4n7\x8fyc\xfe>\xe3\x0c\x91\xc1\xa7\xa2\x9aRa\x8an\xfd\x9b\xe3\x8b\xbf\xcc\x8e<\x19\xb6\x83\x98Uk\xdfW\x9d\xe2\xc6\xf8]\x90\x9eT\xf6a\x97&=\x92\x07\xc5\xb2\xa62\xf43\xbaL,\xac\xde\x82\x0e\x83os \x07\xfc\x8elAs6\xb8\x9e8\x196\xbc\xf25\xdbX]7\xfcc*k:\xe4\xecrp\x07\x98\x06\x97\x19\x100\\D\x9er\x88\xfe\xa2-(\xd7_\x00\x1a\xdbo\x95\x13mn\x98b\xfb:i\x9fc\x98\xd2(\xd53\x99K\xa0\xbfP\xc2\x8a,\x82\x92VI')
Decrypted Data: b'